In [1]:
import datetime as dt
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import cartopy.feature
from cartopy.util import add_cyclic_point
import cartopy.crs as ccrs
import xarray as xr
import math
import netCDF4
#from get_ellipse_metrics import get_emetrics_max_min
from fitEllipse2_new import fitEllipseContour
from geopy.distance import great_circle
from matplotlib.patches import Polygon
from matplotlib import cm #colormaps!
import copy
import os
import scipy.stats
import pickle


In [2]:
#Open data related pickles

infile = open("ephi_wind10_1.p", 'rb') 
ephi_wind10 = pickle.load(infile)
infile.close()


In [3]:
ephi_wind10[2]

array([21.8586216 , 21.34733582, 20.07185745, 21.8989563 , 20.56524467,
       21.57521629, 21.64982414, 23.06637001, 22.30568886, 22.48169899,
       22.03869629, 22.91432762, 23.70433617, 22.32229614, 22.96229744,
       23.65891075, 22.35871887, 24.54321289, 22.4387722 , 24.57108116,
       22.79409981, 22.85801125, 23.80835533, 24.92560577, 23.56970215,
       24.4727478 , 25.16825867, 25.39844704, 25.83750153, 26.45423317,
       27.83402061, 27.74610138, 30.25515175, 29.67618561, 32.49433899,
       32.3411293 , 33.38887405, 34.57536697, 33.30211258, 35.63473129,
       34.97777939, 36.1896553 , 36.27329636, 36.53449631, 36.53544998,
       37.52407455, 37.00932693, 37.02437592, 38.42898941, 39.18457413,
       39.83631516, 40.1224556 , 38.59091568, 38.69313431, 37.38595581,
       36.76436996, 35.79022217, 36.55257797, 37.01012039, 37.01793671,
       35.788311  , 36.85150146, 35.78683853, 36.03741455, 35.68234253,
       35.37300491, 36.71397781, 36.20699692, 35.36317444, 33.85

In [4]:
#loop line and print "i" if year has minimum value less than the SSW standard ... probably won't work.
for i in range(len(ephi_wind10[:])):
    yearmin = np.nanmin(ephi_wind10[i])
    #yeartest = np.where(ephi_wind10[i] == yearmin)
    #low = int(ephi_wind10[i,int(yeartest[0])])
    #high = int(ephi_wind10[i,int(yeartest[0])-40])
    #test = abs((low - high)/100)
    #print(test)
    #if test >= 0.75:
        #print(i)
    if yearmin < 11.8:
        print(i)
    else: 
        continue

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [8]:
##new testing with SSW quals
SSWvals = np.empty((14,88))
SSWvals[:] = np.nan

##I am going to hard code these values just to make sure I am inputing them correctly

SSWvals[0,:] = ephi_wind10[0, 388:476] 
SSWvals[1,:] = ephi_wind10[1, 240:328]
SSWvals[2,:] = ephi_wind10[2, 60:148]
SSWvals[3,:] = ephi_wind10[3, 140:228]
SSWvals[4,:] = ephi_wind10[4, 92:180]
SSWvals[5,:] = ephi_wind10[6, 152:240]
SSWvals[6,:] = ephi_wind10[7, 288:376]
SSWvals[7,:] = ephi_wind10[8, 280:368]
SSWvals[8,:] = ephi_wind10[9, 164:252]
SSWvals[9,:] = ephi_wind10[10, 232:320]
SSWvals[10,:] = ephi_wind10[10, 400:488]
SSWvals[11,:] = ephi_wind10[13, 96:184]
SSWvals[12,:] = ephi_wind10[18, 240:328]
SSWvals[13,:] = ephi_wind10[19, 76:164]

In [9]:
SSWvals[0]

array([26.09321594, 27.41047668, 26.70251274, 26.71653748, 29.15062904,
       28.95764732, 29.62112999, 28.85448265, 28.86714172, 24.96862221,
       23.92280006, 20.33211708, 19.83315659, 18.56147575, 18.05990028,
       19.00487709, 19.30123138, 18.37248802, 18.65393639, 16.50673485,
       16.0757637 , 13.60257721, 12.04218006, 11.15838909, 10.48897552,
        8.79548359,  8.3131218 ,  7.62576866,  8.14530277,  7.48495674,
        8.39905643,  8.74608326, 10.04329681,  9.7735548 , 10.06551647,
        9.92773533, 10.00719452,  4.58077192,  3.00518012,  3.01435089,
        2.85830498,  1.86384904,  4.88291216,  3.63533807,  3.44318271,
        3.36721158,  1.3307966 ,  3.6702776 ,  2.78450775,  1.4433111 ,
        0.45918465,  0.9321785 , -1.5673269 , -2.10103226, -2.72683716,
       -2.51193881, -3.1698525 , -3.63670564, -3.35620236, -1.97345102,
       -2.12510133, -1.55688989, -0.51812619,  1.50834656,  1.50243509,
        2.07805896,  3.46992493,  3.2756443 ,  5.09286737,  4.49

In [11]:
##list for minimum vlaues from each SSW period

maxs = []
mins = []

quantchange = []

for i in range(len(SSWvals[:])):
    mn = np.nanmin(SSWvals[i])
    mins.append(mn)
    
    mx = np.nanmax(SSWvals[i])
    maxs.append(mx)
    
    qt = (mx-mn)/100.
    quantchange.append(qt)

mins

[-3.6367056369781494,
 -6.722714900970459,
 -1.6213722229003906,
 -1.4361038208007812,
 -7.872087001800537,
 -13.155261039733887,
 -4.870993137359619,
 -10.081542015075684,
 -31.91271209716797,
 -4.047924041748047,
 -2.359975814819336,
 -9.391375541687012,
 -25.468292236328125,
 -9.220104217529297]

In [12]:
##what about just quantiles for SSW years?
SSW= np.empty((13,488))
SSW[:] = np.nan

classif = [0,1,2,3,4,6,7,8,9,10,13,18,19]

for i in range(len(SSW[:])):
    #print(classif[i])
    SSW[i] = ephi_wind10[classif[i]]



#wind10 = np.nanmean(SSW,axis=0)

##quantile list appends
uwmin = []
uw10 = []
uw25 = []
uw50 = []
uw75 = []
uw90 = []
uwmax = []

#calculation
for i in range(len(SSW[:])):
    mn = np.nanmin(SSW[i])
    uwmin.append(mn)
    
    u10 = np.nanpercentile(SSW[i],10)
    uw10.append(u10)
    
    u25 = np.nanpercentile(SSW[i],25)
    uw25.append(u25)
    
    u50 = np.nanpercentile(SSW[i],50)
    uw50.append(u50)
    
    u75 = np.nanpercentile(SSW[i],75)
    uw75.append(u75)
    
    u90 = np.nanpercentile(SSW[i],90)
    uw90.append(u90)
    
    mx = np.nanmax(SSW[i])
    uwmax.append(mx)

uwmin = np.mean(uwmin)
uw10 = np.mean(uw10)
uw25 = np.mean(uw25)
uw50 = np.mean(uw50)
uw75 = np.mean(uw75)
uw90 =np.mean(uw90)
uwmax = np.mean(uwmax)

quants = [uwmin,uw10,uw25,uw50,uw75,uw90,uwmax]

quants

[-10.365312778032743,
 -0.07869632484821067,
 9.077052869857885,
 21.65438725398137,
 33.27891309444721,
 41.72669695340671,
 50.963873643141525]

In [13]:
##same for normal years? compare??
##quantile list appends
uwmin = []
uw10 = []
uw25 = []
uw50 = []
uw75 = []
uw90 = []
uwmax = []

#calculation
for i in range(len(ephi_wind10[:])):
    mn = np.nanmin(ephi_wind10[i])
    uwmin.append(mn)
    
    u10 = np.nanpercentile(ephi_wind10[i],10)
    uw10.append(u10)
    
    u25 = np.nanpercentile(ephi_wind10[i],25)
    uw25.append(u25)
    
    u50 = np.nanpercentile(ephi_wind10[i],50)
    uw50.append(u50)
    
    u75 = np.nanpercentile(ephi_wind10[i],75)
    uw75.append(u75)
    
    u90 = np.nanpercentile(ephi_wind10[i],90)
    uw90.append(u90)
    
    mx = np.nanmax(ephi_wind10[i])
    uwmax.append(mx)

uwmin = np.mean(uwmin)
uw10 = np.mean(uw10)
uw25 = np.mean(uw25)
uw50 = np.mean(uw50)
uw75 = np.mean(uw75)
uw90 =np.mean(uw90)
uwmax = np.mean(uwmax)

quants = [uwmin,uw10,uw25,uw50,uw75,uw90,uwmax]

quants

[-9.104625634849072,
 2.545096379294992,
 12.896112109224003,
 25.39775233268738,
 36.65053298473358,
 44.60931394577027,
 53.90324668884277]

In [14]:
#loop line and print "i" if year has minimum value less than the SSW standard ... probably won't work.
#... RETRYING WITH NEW QUANTILES

for i in range(len(ephi_wind10[:])):
    yearmin = np.nanmin(ephi_wind10[i])
    #yeartest = np.where(ephi_wind10[i] == yearmin)
    #low = int(ephi_wind10[i,int(yeartest[0])])
    #high = int(ephi_wind10[i,int(yeartest[0])-40])
    #test = abs((low - high)/100)
    #print(test)
    #if test >= 0.75:
        #print(i)
    if yearmin < -9.10:
        print(i)
    else: 
        continue
        
##ALMOST CAPTURES CORRECT YEARS-- NEGLECTS 2000-2004, 2007 and 2010, SMALLER
#INCLUES 2005?

print(mins)

5
6
8
9
13
14
16
18
19
[-3.6367056369781494, -6.722714900970459, -1.6213722229003906, -1.4361038208007812, -7.872087001800537, -13.155261039733887, -4.870993137359619, -10.081542015075684, -31.91271209716797, -4.047924041748047, -2.359975814819336, -9.391375541687012, -25.468292236328125, -9.220104217529297]
